In [22]:
import pandas as pd 
import numpy as np 
from utils import get_predictions
import torch.nn.functional as F
from dataset import get_data,get_data2



import torch
from sklearn import metrics
import torch.nn as nn
import torch.optim as optim
from dataset import get_data
from torch.utils.data import DataLoader

In [20]:
from tqdm import tqdm

In [54]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


In [5]:
train_ds, val_ds, test_ds, test_ids = get_data()

In [6]:
train_loader = DataLoader(train_ds, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=1024)
val_loader = DataLoader(val_ds, batch_size=1024)

In [7]:


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'



class NN(nn.Module):

    def __init__(self, input_size):
        super(NN, self).__init__()

        self.net = nn.Sequential(
            nn.BatchNorm1d(input_size),
            nn.Linear(input_size, 50),
            nn.ReLU(inplace=True),
            nn.Linear(50, 1)
        )

    def forward(self, x):
        return torch.sigmoid(self.net(x)).view(-1)


model = NN(input_size=200)
optimizer = optim.Adam(model.parameters(), lr=3e-4, weight_decay= 1e-4)
loss_fn = nn.BCELoss()


#x, y = next(iter(train_loader))
#print(x.shape)


In [8]:
for epoch in range(15):
    
    proba, true = get_predictions(val_loader,model) 
    print(f"VALIDATION ROC: {metrics.roc_auc_score(true, proba)}")
    for batch_idx, (data,targets) in enumerate(train_loader) :
       
        
         
        
        scores = model(data)
        #print(scores.shape)
        
        loss = loss_fn(scores,targets)
        #print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        

VALIDATION ROC: 0.5256196675870406
VALIDATION ROC: 0.7934739468686858
VALIDATION ROC: 0.8389242673676187
VALIDATION ROC: 0.849962244291963
VALIDATION ROC: 0.8523371012460422
VALIDATION ROC: 0.8529505313635765
VALIDATION ROC: 0.8534267183434351
VALIDATION ROC: 0.8534896433624757
VALIDATION ROC: 0.8535260971015036
VALIDATION ROC: 0.8536874430362436
VALIDATION ROC: 0.8539320724941917
VALIDATION ROC: 0.8539490384948418
VALIDATION ROC: 0.8540647841666971
VALIDATION ROC: 0.8541401994502278
VALIDATION ROC: 0.8542379272363585


In [9]:
class NN(nn.Module):

    def __init__(self, input_size,hidden_dim):
        super(NN, self).__init__()
        
        self.bn = nn.BatchNorm1d(input_size)
        self.fc1 = nn.Linear(1,hidden_dim)
        self.fc2 = nn.Linear(input_size*hidden_dim,1)
        
        #-------------------------------------------
        
        
        """"self.net = nn.Sequential(
            nn.BatchNorm1d(input_size),
            nn.Linear(input_size, 50),
            nn.ReLU(inplace=True),
            nn.Linear(50, 1)
        )"""

    def forward(self, x):
        
        Batch_size = x.shape[0]
        x = self.bn(x)
        # le shape actuel du shape est : (Batch_size,200)
        x = x.view(-1,1)
        x = F.relu(self.fc1(x)).reshape(Batch_size,-1)
        # Batch_size,200*hidden_dim
        return torch.sigmoid(self.fc2(x)).view(-1)
        # maintenant on aura Batch_size*200,1
        #return torch.sigmoid(self.net(x)).view(-1)


In [10]:
model2 = NN(input_size=200,hidden_dim=16)
optimizer = optim.Adam(model2.parameters(), lr=3e-4, weight_decay= 1e-4)
loss_fn = nn.BCELoss()

In [11]:
for epoch in range(15):
    
    proba, true = get_predictions(val_loader,model2) 
    print(f"VALIDATION ROC: {metrics.roc_auc_score(true, proba)}")
    for batch_idx, (data,targets) in enumerate(train_loader) :
       
        
         
        
        scores = model2(data)
        #print(scores.shape)
        
        loss = loss_fn(scores,targets)
        #print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

VALIDATION ROC: 0.4459659280910186
VALIDATION ROC: 0.8008896321497987
VALIDATION ROC: 0.8441723553679619
VALIDATION ROC: 0.8517911241296556
VALIDATION ROC: 0.8581167673796699
VALIDATION ROC: 0.8637933522590912
VALIDATION ROC: 0.8684279404806364
VALIDATION ROC: 0.8723297305271065
VALIDATION ROC: 0.8750814444658598
VALIDATION ROC: 0.8770950311429037
VALIDATION ROC: 0.8787539583967889
VALIDATION ROC: 0.8798535266056288
VALIDATION ROC: 0.8808614519396774
VALIDATION ROC: 0.8816228460632929
VALIDATION ROC: 0.8822488953186545


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
test.shape

(200000, 201)

In [5]:
col_names = [f'var_{i}' for i in range(200)]

In [6]:
for col in tqdm(col_names):
    count = test[col].value_counts()
    uniques = count.index[count == 1]
    test[col+"_u"] = test[col].isin(uniques)
test["has_unique"] = test[[col+'_u' for col in col_names]].any(axis=1)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:06<00:00, 32.83it/s]


In [7]:
test["has_unique"]

0         False
1         False
2         False
3          True
4         False
          ...  
199995     True
199996     True
199997    False
199998    False
199999     True
Name: has_unique, Length: 200000, dtype: bool

In [8]:
test['has_unique'].sum()
# we have 100000 real data and the rest are fake 

100000

In [9]:
real_test = test.loc[test["has_unique"], ["ID_code"] + col_names]
fake_test = test.loc[~test["has_unique"], ["ID_code"] + col_names]

In [10]:
train_and_test = pd.concat([train, real_test], axis=0)

In [11]:
for col in tqdm(col_names):
    count = train_and_test[col].value_counts().to_dict()
    train_and_test[col+"_unique"] = train_and_test[col].apply(
        lambda x: 1 if count[x] == 1 else 0).values
    fake_test[col+"_unique"] = 0 
    #print(count)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [01:05<00:00,  3.05it/s]


In [12]:
train_and_test

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190_unique,var_191_unique,var_192_unique,var_193_unique,var_194_unique,var_195_unique,var_196_unique,var_197_unique,var_198_unique,var_199_unique
0,train_0,0.0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,0,0,0,0,0,0,0,0,0,0
1,train_1,0.0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,0,0,0,0,0,0,0,0,0,0
2,train_2,0.0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,0,0,0,0,0,0,0,0,0,0
3,train_3,0.0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,0,0,0,0,0,0,0,0,0,0
4,train_4,0.0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,0,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199986,test_199986,NaN,19.2884,-2.8384,11.9149,6.6611,12.3112,12.9244,5.6492,16.0449,...,0,0,0,0,0,0,0,0,0,0
199993,test_199993,NaN,14.6764,-8.1066,7.1167,2.4138,10.3845,-11.9327,4.7563,16.0455,...,0,0,0,0,0,0,0,0,0,1
199995,test_199995,NaN,13.1678,1.0136,10.4333,6.7997,8.5974,-4.1641,4.8579,14.7625,...,1,0,0,0,1,0,0,0,0,1
199996,test_199996,NaN,9.7171,-9.1462,7.3443,9.1421,12.8936,3.0191,5.6888,18.8862,...,1,0,0,0,0,0,0,0,0,1


In [13]:
real_test = train_and_test[train_and_test["ID_code"].str.contains("test")].copy()
real_test.drop(["target"], axis=1, inplace=True)
train = train_and_test[train_and_test["ID_code"].str.contains("train")].copy()

In [14]:
test = pd.concat([real_test, fake_test], axis=0)

In [15]:
train.to_csv("new_shiny_train.csv", index=False)
test.to_csv("new_shiny_test.csv", index=False)

In [109]:
test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190_unique,var_191_unique,var_192_unique,var_193_unique,var_194_unique,var_195_unique,var_196_unique,var_197_unique,var_198_unique,var_199_unique
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,0,0,0,0,0,0,0,0,0,1
7,test_7,17.3035,-2.4212,13.3989,8.3998,11.0777,9.6449,5.9596,17.8477,-4.8068,...,0,0,0,0,0,0,0,0,1,0
11,test_11,10.6137,-2.1898,8.9090,3.8014,13.8602,-5.9802,5.5515,15.4716,-0.1714,...,0,0,0,0,0,0,1,0,0,1
15,test_15,14.8595,-4.5378,13.6483,5.6480,9.9144,1.5190,5.0358,13.4524,-2.5419,...,0,0,1,0,0,0,0,0,0,1
16,test_16,14.1732,-5.1490,9.7591,3.7316,10.3700,-21.9202,7.7130,18.8749,0.4680,...,0,0,0,0,0,0,0,0,0,0


In [16]:
train_ds, val_ds, test_ds, test_ids = get_data2()

In [17]:
train_loader = DataLoader(train_ds, batch_size=1024, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=1024)
val_loader = DataLoader(val_ds, batch_size=1024)

In [23]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_dim):
        super(NN, self).__init__()
        self.bn = nn.BatchNorm1d(input_size)
        self.fc1 = nn.Linear(2, hidden_dim)
        self.fc2 = nn.Linear(input_size//2*hidden_dim, 1)

    def forward(self, x):
        N = x.shape[0]
        x = self.bn(x)
        orig_features = x[:, :200].unsqueeze(2) # (N, 200, 1)
        new_features = x[:, 200:].unsqueeze(2) # (N, 200, 1)
        x = torch.cat([orig_features, new_features], dim=2) # (N, 200, 2)
        x = F.relu(self.fc1(x)).reshape(N, -1) # (N, 200*hidden_dim)
        return torch.sigmoid(self.fc2(x)).view(-1)


In [25]:
model3 = NN(input_size=400, hidden_dim=100)
optimizer = optim.Adam(model3.parameters(), lr=2e-3, weight_decay=1e-4)
loss_fn = nn.BCELoss()

In [26]:
for epoch in range(15):
    
    proba, true = get_predictions(val_loader,model3) 
    print(f"VALIDATION ROC: {metrics.roc_auc_score(true, proba)}")
    for batch_idx, (data,targets) in enumerate(train_loader) :
       
        
         
        
        scores = model3(data)
        #print(scores.shape)
        
        loss = loss_fn(scores,targets)
        #print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

VALIDATION ROC: 0.6884469696969696
VALIDATION ROC: 0.9308712121212122
VALIDATION ROC: 0.9183238636363635
VALIDATION ROC: 0.928503787878788
VALIDATION ROC: 0.9377367424242424
VALIDATION ROC: 0.9386837121212122
VALIDATION ROC: 0.9261363636363635
VALIDATION ROC: 0.9341856060606061
VALIDATION ROC: 0.9363162878787878
VALIDATION ROC: 0.9308712121212122
VALIDATION ROC: 0.9427083333333333
VALIDATION ROC: 0.9356060606060606
VALIDATION ROC: 0.9325284090909092
VALIDATION ROC: 0.935842803030303
VALIDATION ROC: 0.9382102272727272


In [27]:
def get_submission(model, loader, test_ids):
    all_preds = []
    model.eval()
    with torch.no_grad():
        for x,y in loader:
            #print(x.shape)
            x = x
            score = model(x)
            prediction = score.float()
            all_preds += prediction.tolist()

    model.train()

    df = pd.DataFrame({
        "ID_code" : test_ids.values,
        "target" : np.array(all_preds)
    })

    df.to_csv("sub.csv", index=False)

In [28]:
get_submission(model3, test_loader, test_ids)

torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024, 400])
torch.Size([1024